<b>
<div style="text-align: center;">
<span style="font-size: 200%;">方策反復(Policy iteration)</span>
</div>
</b>

<br>

<b>
<div style="text-align: right;">
    <span style="font-size: 150%;">
        2020/1/10 
    </span><br>
    <span style="font-size: 150%;">
        Masaya Mori
    </span>
</div>
</b>

# 目的

<span style="font-size: 120%;"> 今回は方策反復によって，方策の更新を行う． </span>

## 方策反復(Policy iteration)とは

<span style="font-size: 120%;"> 方策反復とは，
    <span style="color: red"> **反復方策評価(Iterative Policy Evaluation)**</span>
    によって算出した状態価値から
    <span style="color: red"> **方策改善(Policy improvement)**</span>
    により，ある方策下での行動報酬が最大となるように方策を更新していくことである．</span>
<span style="font-size: 120%;"> より分かりやすく説明すると， </span> </br>

1. <span style="font-size: 120%;"> ある方策$\pi$が存在するとき </span> </br>
1. <span style="font-size: 120%;"> その方策$\pi$に従ったときの状態価値を算出し(反復方策評価) </span> </br>
1. <span style="font-size: 120%;"> その状態時における最適な方策$\pi^*$を算出する(方策改善) </span> </br>

<span style="font-size: 120%;"> ということであり，1～3を繰り返すことを
    <span style="color: red">**方策反復**(Policy iteration)</span>
    という．</span> 

## 反復方策評価(Iterative Policy Evaluation)

<span style="font-size: 120%;"> これは，今まで再帰処理によって状態価値を求めていたけど，時間がかかるから反復処理で状態価値関数の近似を試みるってだけな話です．前回実装したんで，気になる方はこちらを参照． </span>

## 方策改善(Policy improvement)

<span style="font-size: 120%;"> 今回は割とこれがメインです． </span> </br>
<span style="font-size: 120%;"> 初めに，次の図を参照してほしい(最適方策)． </span> </br>
<span style="font-size: 120%;"> ※ 方策改善の1つに最適方策がある感じ </span> </br>

![方策改善](policy_improvement.jpg)

<span style="font-size: 120%;"> ある状態ss'が与えられた時、取り得る行動（黒丸）が4つに分岐しています。これらのどれが一番良い行動なのかを決めるのがこの章の目的です。</span>

<span style="font-size: 120%;"> 行動aをとった結果は、状態遷移確率Pass′Pss′aを経て、報酬Rass′Rss′aおよび次の状態の価値関数Vπ(s)Vπ(s)で表されます。それならば、なるべく多い報酬と次の状態の価値を持つものを最適な行動とするのが自然です。</span>

<span style="font-size: 120%;"> すなわち、灰色で囲んだ黒丸以下のユニット∑s′Pass′[Rass′+γVπ(s′)]∑s′Pss′a[Rss′a+γVπ(s′)]が最も高い行動が状態ssでの最適な方策π′(s)π′(s)になります。</span>

π′(s)=arg maxa∑s′Pass′[Rass′+γVπ(s′)]
π′(s)=arg maxa∑s′Pss′a[Rss′a+γVπ(s′)]
<span style="font-size: 120%;"> ただし、arg maxa(x)arg maxa(x)は、xが最大となるようなaを返す関数です。数式よりも図でイメージで理解した方が早いですね。</span>

注
ここでは、最適な関数を決定論的な関数としています。つまり最適な関数なので、確率的な方策のように「a1a1が90%, a2a2が5%...」とするのではなく、「最適な関数はa1a1」と断言しています。ただし、確率的な方策にも、今回の議論を当てはめることができます。（Sutton本 P.102にその記述があります。）

方策改善は，今までのような『upが0.25でdownが0.25で...』という感じではなく，『上下左右の状態価値を見たら，右の価値が一番高いから100%で右行きます!』ってな感じ。だから，一番最初の時点で『状態$s_1$の時は100%下行きます』的なのを決めとかなければならない．最初のはランダムで決めておけば良いかと．

## アルゴリズム

## Pythonでの実装

In [5]:
import numpy as np

In [6]:
# ---------- Step1 : 初期化 ----------
N = 1000
Gamma = 0.9

# 格子世界の大きさ
x_axis = 5
y_axis = 5

# 現在の価値関数と更新後の価値関数の初期化
V      = np.zeros(x_axis * y_axis)
V_next = np.zeros([N, x_axis * y_axis])

# 現在の方策と更新後の方策の初期化
pi      = np.zeros(x_axis * y_axis, dtype="int")
pi_next = np.zeros([N, x_axis * y_axis], dtype="int")

# 格子世界の構築
stage = [[0,4],[1,4],[2,4],[3,4],[4,4],
         [0,3],[1,3],[2,3],[3,3],[4,3],
         [0,2],[1,2],[2,2],[3,2],[4,2],
         [0,1],[1,1],[2,1],[3,1],[4,1],
         [0,0],[1,0],[2,0],[3,0],[4,0]]

In [7]:
# エージェントの設計
# エージェントには，『状態』『行動』『報酬』『方策』を持たせている

class Agent():    
    
    def __init__(self):
        # 行動aの宣言。引数で使うので、初めに宣言する。
        self.actions = [[0,1],[0,-1],[-1,0],[1,0]] # up,down,left,right
    
        self.position = []
        
        # 方策は『各状態時での行動』を持たせている
        # 0～3は『up,down,left,right』を示している
        self.pi = np.random.randint(0,4,25)
        
    def get_actions(self):
        return self.actions
    
    def set_pos(self,now_pos):
        # 現在地の更新
        self.position = now_pos
        
    def get_pos(self):
        # 現在地の取得
        return self.position
    
    def set_pi(self,new_pi):
        self.pi = new_pi
    
    def get_pi(self):
        return self.pi
    
    def move(self,action):
        # 行動後の位置の取得
        if self.get_pos() == [1,4]: # 10ptゾーン
            next_pos = [1,0]
        elif self.get_pos() == [3,4]: # 5ptゾーン
            next_pos = [3,2]
        else :
            next_pos = [self.get_pos()[0] + action[0], self.get_pos()[1] + action[1]]
        
        # 境界線外に出ている時の処理
        if 0 > next_pos[0] or next_pos[0] > 4 or 0 > next_pos[1] or next_pos[1] > 4:
            next_pos = self.get_pos()
            
        self.set_pos(next_pos)
        
    def reward(self,state,action):
        if state == [1,4]:
            return 10
        
        if state == [3,4]:
            return 5
        
        if state[1] == 4 and action == [0,1]:
            return -1
        elif state[1] == 0 and action == [0,-1]:
            return -1
        elif state[0] == 0 and action == [-1,0]:
            return -1
        elif state[0] == 4 and action == [1,0]:
            return -1
        else :
            return 0

In [8]:
##### ここから方策反復(Policy iteration) #####
#        アルゴリズム通りに実装します       #

agent = Agent()

loop_count = 1
pi_next[loop_count-1] = agent.get_pi()

policyStable = False
while(policyStable == False):
    V = np.zeros(x_axis * y_axis)
    # ---------- Step2 : 反復方策評価 ----------
    while(True):
        for i in range(y_axis):
            for j in range(x_axis):
                pos = i*5+j
                delta = 0
                agent.set_pos(stage[pos])
                v = V[pos]
                agent.move(agent.get_actions()[agent.get_pi()[pos]])
                V[pos] = (agent.reward(stage[pos], agent.get_actions()[agent.get_pi()[pos]])
                       + (Gamma * V[stage.index(agent.get_pos())]))
                delta = max(delta,abs(v - V[pos]))
        if delta < 1e-5:
            break
    V_next[loop_count] = V

    
    # ---------- Step3 : 方策改善 ----------
    pi = agent.get_pi()
    temp = np.zeros(len(agent.get_actions()))
    for a in range(x_axis*y_axis):
        for b, action in enumerate(agent.get_actions()):
            agent.set_pos(stage[a])
            agent.move(action)
            temp[b] = (agent.reward(stage[a], action)
                    + (Gamma * V[stage.index(agent.get_pos())]))
        pi_next[loop_count,a] = np.argmax(temp)
    agent.set_pi(pi_next[loop_count])
    
    if np.all(pi == pi_next[loop_count]) :
        policyStable = True

    loop_count += 1


## 結果

結果が正しいことを確認しました

### 状態価値の変化

In [9]:
print(V_next[0].reshape([5,5]))
print(V_next[1].reshape([5,5]))
print(V_next[2].reshape([5,5]))
print(V_next[3].reshape([5,5]))
print(V_next[4].reshape([5,5]))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[ 0.    10.     0.     5.    -1.   ]
 [ 0.     0.     0.     4.5    4.05 ]
 [ 0.     0.     0.     0.     3.645]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.    -1.    -1.     0.   ]]
[[21.97748527 24.41942809 21.97748528 18.45014107 16.60512696]
 [19.77973674 21.97748528 14.94460119 16.60512696 14.94461426]
 [17.80176307 19.77973675 13.45014107 14.94461426 13.45015284]
 [16.02158676 17.80176308 12.10512696 13.45015284 12.10513755]
 [14.41942809 16.02158677 10.89461426 12.10513755 10.8946238 ]]
[[21.97748527 24.41942809 21.97748528 18.45014107 16.60512696]
 [19.77973674 21.97748528 19.77973675 16.60512696 14.94461426]
 [17.80176307 19.77973675 17.80176308 14.94461426 13.45015284]
 [16.02158676 17.80176308 16.02158677 13.45015284 12.10513755]
 [14.41942809 16.02158677 14.41942809 12.10513755 10.8946238 ]]
[[21.97746054 24.41941186 21.97747068 19.41941186 17.47747068]
 [19.77971449 21.977470

### 方策の変化

In [10]:
# (0,1,2,3)は(up,down,left,right)

print(pi_next[0].reshape([5,5]))
print(pi_next[1].reshape([5,5]))
print(pi_next[2].reshape([5,5]))
print(pi_next[3].reshape([5,5]))
print(pi_next[4].reshape([5,5]))

[[1 3 3 0 3]
 [1 1 2 0 2]
 [0 1 2 2 0]
 [3 1 3 1 1]
 [0 0 1 1 0]]
[[3 0 2 0 2]
 [0 0 3 0 2]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[3 0 2 0 2]
 [3 0 0 0 0]
 [3 0 2 0 0]
 [3 0 2 0 0]
 [3 0 2 0 0]]
[[3 0 2 0 2]
 [3 0 0 2 0]
 [3 0 0 2 0]
 [3 0 0 2 0]
 [3 0 0 2 0]]
[[3 0 2 0 2]
 [3 0 0 2 2]
 [3 0 0 0 2]
 [3 0 0 0 2]
 [3 0 0 0 2]]


## 参考資料

[今さら聞けない強化学習（6）：反復法による最適方策](https://qiita.com/triwave33/items/59768d14da38f50fb76c#%E6%96%B9%E7%AD%96%E5%8F%8D%E5%BE%A9) </br>
[強化学習：再帰処理と反復処理](https://shirakonotempura.hatenablog.com/entry/2019/01/31/070932)